#### **Ingesta del archivo "language_role.json"**


In [0]:
dbutils.widgets.text("environment","")
var_environment = dbutils.widgets.get("environment")

In [0]:
dbutils.widgets.text("file_date","2024-12-16")
var_file_date = dbutils.widgets.get("file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

##### Librerías

In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType
from pyspark.sql.functions import lit, current_timestamp

##### Esquema

In [0]:
language_role_schema = StructType([
    StructField("roleId",IntegerType(),False),
    StructField("languageRole",StringType(),True)
])

##### Leer fichero JSON

In [0]:
df_language_role = spark.read.option("multiline",True).json(f"{bronze_folder_path}/{var_file_date}/language_role.json")

##### Renombrar y añadir columnas

In [0]:
df_language_role_final = add_ingestion_date(df_language_role).withColumnsRenamed({"roleId":"role_id","languageRole":"language_role"}) \
                                         .withColumn("environment",lit(var_environment)).withColumn("file_date",lit(var_file_date))

##### Escribir en Silver

In [0]:
#df_language_role_final.write.mode("overwrite").parquet(f"{silver_folder_path}/language_roles")
df_language_role_final.write.mode("overwrite").format("delta").saveAsTable("movie_silver.language_roles")

In [0]:
dbutils.notebook.exit("Success")